In [5]:
# -*- coding: utf-8 -*-
"""
@author: Gemini (Final Corrected Version 5.12)
@date: 2025-10-10
@description: 
    此腳本為「交屋稅費分算計算機」的功能更新版本。
    v5.12 更新日誌:
    1.  [重大功能] 房屋現值欄位(B9)現已支援 "總現值-免稅現值" 格式輸入，
                 例如 "13272200-277300"。
    2.  [重大優化] 房屋稅備註(F17)會根據 B9 的輸入格式，自動顯示詳細的
                 (總現值-免稅現值)計算過程，讓稅額來源一目了然。
    v5.11 更新日誌:
    1.  [重大調整] 移除【工作表保護】功能，以方便使用者後續自行加入 VBA 巨集。
"""
import openpyxl
from openpyxl.styles import PatternFill, Font, Alignment, Border, Side
from openpyxl.worksheet.datavalidation import DataValidation
from openpyxl.comments import Comment


def apply_styles_to_range(ws, cell_range, styles):
    """輔助函式：對指定的儲存格範圍一次性套用多種樣式。"""
    valid_styles = ['font', 'fill', 'alignment', 'border', 'number_format']
    rows = ws[cell_range]
    if not isinstance(rows, tuple):
        rows = ((rows,),)
    for row in rows:
        for cell in row:
            for style_name, style_obj in styles.items():
                if style_name in valid_styles and style_obj is not None:
                    setattr(cell, style_name, style_obj)


def create_final_workbook_v5_12(filename="交屋稅費計算機(v5.12_房屋現值優化版).xlsm"):
    """建立、格式化並儲存包含複雜公式與樣式的稅費計算機 Excel 檔案。"""
    wb = openpyxl.Workbook()
    ws = wb.active
    ws.title = "專業稅費分算計算機"

    # --- 設定檔案的核心屬性 ---
    wb.properties.creator = "陳定康"
    wb.properties.title = "不動產買賣稅費分算計算機"
    wb.properties.description = "此工具用於精確計算不動產買賣過程中，買賣雙方應分攤之地價稅、房屋稅及其他相關費用。"

    # --- 樣式定義 ---
    fill_yellow_input = PatternFill(
        start_color='FFFFE0', end_color='FFFFE0', fill_type='solid')
    fill_blue_input = PatternFill(
        start_color='EAF1FF', end_color='EAF1FF', fill_type='solid')
    fill_title_green = PatternFill(
        start_color='D7E9D1', end_color='D7E9D1', fill_type='solid')
    fill_header_grey = PatternFill(
        start_color='E7E6E6', end_color='E7E6E6', fill_type='solid')
    fill_result_orange = PatternFill(
        start_color='FFF2E6', end_color='FFF2E6', fill_type='solid')
    fill_period_land = PatternFill(
        start_color='E2EFDA', end_color='E2EFDA', fill_type='solid')
    fill_period_house = PatternFill(
        start_color='DEEBF7', end_color='DEEBF7', fill_type='solid')
    font_title = Font(name='微軟正黑體', size=18, bold=True)
    font_section_header = Font(name='微軟正黑體', size=14, bold=True)
    font_body = Font(name='微軟正黑體', size=12)
    font_body_bold = Font(name='微軟正黑體', size=12, bold=True)
    font_red_bold = Font(name='微軟正黑體', size=12, bold=True, color="C00000")
    font_final_result = Font(
        name='微軟正黑體', size=16, bold=True, color="C00000")
    font_note = Font(name='微軟正黑體', size=11, italic=True)
    font_footer = Font(name='微軟正黑體', size=10, color="808080")
    align_center = Alignment(
        horizontal='center', vertical='center', wrap_text=True)
    align_left = Alignment(
        horizontal='left', vertical='center', wrap_text=True)
    align_right = Alignment(
        horizontal='right', vertical='center', wrap_text=True)
    thin_border = Border(left=Side(style='thin'), right=Side(
        style='thin'), top=Side(style='thin'), bottom=Side(style='thin'))
    medium_border = Border(left=Side(style='medium'), right=Side(
        style='medium'), top=Side(style='medium'), bottom=Side(style='medium'))

    # --- 欄寬與列高設定 ---
    for col, width in {'A': 30, 'B': 20, 'C': 20, 'D': 16, 'E': 20, 'F': 55, 'G': 20, 'I': 90}.items():
        ws.column_dimensions[col].width = width
    for i in range(1, 40):
        ws.row_dimensions[i].height = 32
    for row, height in {1: 50, 15: 45, 16: 55, 17: 55, 30: 55}.items():
        ws.row_dimensions[row].height = height
    ws.row_dimensions[3].height = 130
    ws.row_dimensions[9].height = 130

    # --- 輔助計算區 ---
    ws.column_dimensions['AA'].hidden = True
    ws.column_dimensions['AB'].hidden = True
    ws['AA1'] = '=IFERROR(IF(LEN(B2)=7, DATE(VALUE(LEFT(B2,3))+1911, VALUE(MID(B2,4,2)), VALUE(RIGHT(B2,2))), IF(LEN(B2)=6, DATE(VALUE(LEFT(B2,2))+1911, VALUE(MID(B2,3,2)), VALUE(RIGHT(B2,2))), "")), "")'
    ws['AA2'] = '=IFERROR(IF(LEN(B3)=7, DATE(VALUE(LEFT(B3,3))+1911, VALUE(MID(B3,4,2)), VALUE(RIGHT(B3,2))), IF(LEN(B3)=6, DATE(VALUE(LEFT(B3,2))+1911, VALUE(MID(B3,3,2)), VALUE(RIGHT(B3,2))), "")), "")'
    ws['AA3'] = '=IFERROR(IF(C7<>"", (LEFT(C7, FIND("/", C7)-1) / MID(C7, FIND("/", C7)+1, LEN(C7))) * 100, B7), B7)'
    ws['AA4'] = '=IF(B4="是", 1, 0)'
    ws['AA5'] = '=IF(AA2="","",AND(DAY(EOMONTH(AA2,0))=DAY(AA2), B4="否"))'
    ws['AA6'] = '=IFERROR(IF(LEN(B23)=7, DATE(VALUE(LEFT(B23,3))+1911, VALUE(MID(B23,4,2)), VALUE(RIGHT(B23,2))), IF(LEN(B23)=6, DATE(VALUE(LEFT(B23,2))+1911, VALUE(MID(B23,3,2)), VALUE(RIGHT(B23,2))), "")), "")'
    ws['AA7'] = '=IFERROR(IF(LEN(B24)=7, DATE(VALUE(LEFT(B24,3))+1911, VALUE(MID(B24,4,2)), VALUE(RIGHT(B24,2))), IF(LEN(B24)=6, DATE(VALUE(LEFT(B24,2))+1911, VALUE(MID(B24,3,2)), VALUE(RIGHT(B24,2))), "")), "")'
    # 【新功能】新增輔助格，解析 B9 儲存格的 "A-B" 格式
    # 解析 A (總現值)
    ws['AA8'] = '=IFERROR(IF(ISNUMBER(FIND("-",B9)), VALUE(LEFT(B9, FIND("-",B9)-1)), B9), B9)'
    # 解析 B (免稅額)
    ws['AA9'] = '=IFERROR(IF(ISNUMBER(FIND("-",B9)), VALUE(MID(B9, FIND("-",B9)+1, LEN(B9))), 0), 0)'
    ws['AA10'] = '=IF(ISNUMBER(B9), B9, AA8-AA9)'  # 計算最終課稅現值 (A-B)

    # --- 核心公式定義 ---
    calc_formulas = {
        'AB1': '=IF(B12<>"", B12, IFERROR(ROUND(B5*B6*(AA3/100)*VLOOKUP(B8,稅率表!$D$2:$E$3,2,FALSE),0),""))',
        # 【公式修改】房屋稅計算公式改為參照新的輔助格 AA10
        'AB2': '=IF(B13<>"", B13, IFERROR(ROUND(AA10*VLOOKUP(B10, 稅率表!$A$2:$B$14, 2, FALSE),0), ""))',
        'AB3': '=IF(AA5, 12, IFERROR(DATE(YEAR(AA2),12,31)-DATE(YEAR(AA2),1,1)+1, ""))',
        'AB4': '=IFERROR(DATE(YEAR(AA2)-IF(MONTH(AA2)<7,1,0),7,1), "")',
        'AB5': '=IFERROR(DATE(YEAR(AA2)-IF(MONTH(AA2)<7,1,0)+1,6,30), "")',
        'AB6': '=IF(AA5, 12, IFERROR(AB5-AB4+1, ""))',
        'AB7': '=IFERROR(EOMONTH(DATE(YEAR(AB5),2,1),0), "")',
        'AB8': '=IFERROR(IF(AA1>DATE(YEAR(AA2),8,31), "賣方", "買方"), "")',
        'AB9': '=IFERROR(IF(AA1>AB7, "賣方", "買方"), "")',
        'AB10': '=IFERROR(IF(AA5, MONTH(AA2), AA2-DATE(YEAR(AA2),1,1)+1-AA4),"")',
        'AB11': '=IFERROR(IF(AA5, 12-MONTH(AA2), DATE(YEAR(AA2),12,31)-AA2+AA4),"")',
        'AB12': '=IFERROR(IF(AA5, IF(MONTH(AA2)>=7, MONTH(AA2)-6, MONTH(AA2)+6), AA2-AB4+1-AA4),"")',
        'AB13': '=IFERROR(IF(AA5, 12 - IF(MONTH(AA2)>=7, MONTH(AA2)-6, MONTH(AA2)+6), AB5-AA2+AA4),"")',
        'AB21': '=IFERROR(IF(AND(B20>0, AA6>=AA2), ROUND(B20 * (AA6-AA2+AA4) / DAY(EOMONTH(AA2,0)), 0), 0), 0)',
        'AB22': '=IFERROR(IF(AND(B21>0, AA7>=AA2), ROUND(B21 * (AA7-AA2+AA4) / DAY(EOMONTH(AA2,0)), 0), 0), 0)',
        'AB23': '=AB21+AB22',
        'AB25': '=IFERROR(IF(AB8="買方", E16, 0) + IF(AB9="買方", E17, 0), 0) + IF(B22<>"", B22, 0)',
        'AB26': '=IFERROR(IF(AB8="賣方", E16, 0) + IF(AB9="賣方", E17, 0) + AB23, 0)',
        'B16': '=IF(AB10<>"", AB10 & IF(AA5, " 月", " 天"), "")',
        'C16': '=IF(AB11<>"", AB11 & IF(AA5, " 月", " 天"), "")',
        'D16': '=AB1',
        'E16': '=IFERROR(IF(AB8="賣方", ROUND(D16*AB11/AB3,0), ROUND(D16*AB10/AB3,0)),0)',
        # 【公式優化】地價稅備註格式統一
        'F16': '=IF(E16=0, "無需找補", "● 總稅額: " & TEXT(D16,"#,##0") & CHAR(10) & "● 找補額: 由【" & IF(AB8="賣方","買方","賣方") & "】補貼納稅人【" & AB8 & "】" & TEXT(E16,"#,##0") & "元")',
        'B17': '=IF(AB12<>"", AB12 & IF(AA5, " 月", " 天"), "")',
        'C17': '=IF(AB13<>"", AB13 & IF(AA5, " 月", " 天"), "")',
        'D17': '=AB2',
        'E17': '=IFERROR(IF(AB9="賣方", ROUND(D17*AB13/AB6,0), ROUND(D17*AB12/AB6,0)),0)',
        # 【公式優化】房屋稅備註，自動顯示(A-B)計算過程
        'F17': '=IF(E17=0, "無需找補", "● 總稅額: " & IF(AA9>0, "(" & TEXT(AA8,"#,##0") & " - " & TEXT(AA9,"#,##0") & ")", TEXT(AA10,"#,##0")) & " × " & TEXT(D10,"0.0%") & " = " & TEXT(D17,"#,##0") & CHAR(10) & "● 找補額: 由【" & IF(AB9="賣方","買方","賣方") & "】補貼納稅人【" & AB9 & "】" & TEXT(E17,"#,##0") & "元")',
        'D25': '=AB23',
        'B29': '=AB25', 'C29': '=AB26',
        'B30': '=IF(C29>B29, "【買方】應支付給【賣方】 NT$ " & TEXT(C29-B29, "#,##0"), IF(B29>C29, "【賣方】應支付給【買方】 NT$ " & TEXT(B29-C29, "#,##0"), "雙方無需找補"))',
        'I2': '=IF(AA2="","", "一、地價稅")',
        'I3': '=IF(AA2="","", CONCATENATE("● 開徵期間：每年 11 月 1 日至 11 月 30 日止。", CHAR(10), "● 納稅義務基準日：每年 8 月 31 日。", CHAR(10), "● 本案課稅年度：民國 ", YEAR(AA2)-1911, " 年 1 月 1 日至 ", YEAR(AA2)-1911, " 年 12 月 31 日。", CHAR(10), "● 繳納義務人：分算後，民國 ", YEAR(AA2)-1911, " 年度開徵之地價稅單將由【", AB8, "】負責繳納。"))',
        'I8': '=IF(AA2="","", "二、房屋稅")',
        'I9': '=IF(AA2="","", CONCATENATE("● 開徵期間：每年 5 月 1 日至 5 月 31 日止。", CHAR(10), "● 納稅義務基準日：每年 2 月之末日。", CHAR(10), "● 本案課稅期間：自民國 ", YEAR(AB4)-1911, " 年 7 月 1 日起至 ", YEAR(AB5)-1911, " 年 6 月 30 日止。", CHAR(10), "● 繳納義務人：分算後，民國 ", YEAR(AB5)-1911, " 年度開徵之房屋稅單將由【", AB9, "】負責繳納。"))',
    }
    for cell_ref, formula in calc_formulas.items():
        ws[cell_ref] = formula

    # --- 介面佈局與靜態文字 ---
    # ... (此處程式碼與 v5.11 基本相同，但 B9 預設值已修改) ...
    ws.merge_cells('A1:F1')
    ws['A1'] = "不動產買賣稅費分算計算機"
    apply_styles_to_range(ws, 'A1', {
                          'font': font_title, 'fill': fill_title_green, 'alignment': align_center})
    ws['A2'] = "權狀登記(過戶)日期"
    ws['A3'] = "房屋點交(交屋)日期"
    ws['A4'] = "是否帶租約交屋"
    apply_styles_to_range(ws, 'A2:A4', {
                          'font': font_body, 'fill': fill_header_grey, 'alignment': align_right})
    ws['B2'] = 1141001
    ws['B3'] = 1141031
    ws['B4'] = "否"

    apply_styles_to_range(ws, 'B2:B4', {
                          'font': font_body, 'fill': fill_yellow_input, 'alignment': align_center})
    dv_lease = DataValidation(type="list", formula1='"是,否"')
    dv_lease.add('B4')
    ws.add_data_validation(dv_lease)
    ws['C2'] = '=IF(AA1<>"", TEXT(AA1, "民國 e 年 m 月 d 日"), "請依左側格式輸入日期")'
    ws['C3'] = '=IF(AA2<>"", TEXT(AA2, "民國 e 年 m 月 d 日"), "請依左側格式輸入日期")'
    ws.merge_cells('C2:D2')
    ws.merge_cells('C3:D3')
    ws.merge_cells('C4:D4')
    apply_styles_to_range(
        ws, 'C2:C3', {'font': font_body, 'alignment': align_left})
    ws['C4'] = '=IF(AA5, "【按月分算模式】", "【按日分算模式】") & " 交屋日歸屬：" & IF(B4="是", "買方", "賣方")'
    apply_styles_to_range(
        ws, 'C4', {'font': font_body_bold, 'alignment': align_left})
    ws.merge_cells('E2:F4')
    ws['E2'] = '=IF(AA1="","",IF(MOD(YEAR(AA1)-1911,2)=1, "【注意】\n民國 " & YEAR(AA1)-1911 & " 年為地價調整年！\n前一年稅單僅供參考。", "【資訊】\n民國 " & YEAR(AA1)-1911 & " 年非地價調整年。"))'
    apply_styles_to_range(
        ws, 'E2', {'font': font_red_bold, 'alignment': align_center})
    ws.merge_cells('A5:A10')
    ws['A5'] = "模式一：詳細資料輸入"
    apply_styles_to_range(ws, 'A5', {
                          'font': font_section_header, 'fill': fill_header_grey, 'alignment': align_center})
    for cell_ref, label in {"E5": "申報地價 (元/m²)", "E6": "土地面積 (m²)", "E7": "權利範圍 (% 或 分數)", "E8": "地價稅率", "E9": "房屋現值 (元)", "E10": "房屋使用情境"}.items():
        ws.merge_cells(f'{cell_ref[0]}{cell_ref[1:]}:F{cell_ref[1:]}')
        ws[cell_ref] = label
        apply_styles_to_range(
            ws, cell_ref, {'font': font_body, 'alignment': align_left})
    for row_num in [5, 6, 8, 9]:
        ws.merge_cells(f'B{row_num}:D{row_num}')
    ws.merge_cells('C7:D7')
    ws.merge_cells('B10:C10')
    apply_styles_to_range(ws, 'B5:B10', {
                          'font': font_body, 'fill': fill_yellow_input, 'alignment': align_center})
    apply_styles_to_range(ws, 'D10', {
                          'font': font_body, 'fill': fill_yellow_input, 'alignment': align_center})
    ws['B5'] = 155091.2
    ws['B6'] = 449
    ws['B7'] = 1.28
    ws['B7'].number_format = '0.00"%"'
    ws['C7'] = ""
    ws['B8'] = "一般用地 (千分之十)"
    # 【介面修改】B9 預設值改為 A-B 格式
    ws['B9'] = '13272200-277300'
    ws['B10'] = "自住用-全國3戶內"
    ws['D10'] = '=IFERROR(VLOOKUP(B10, 稅率表!$A$2:$B$14, 2, FALSE), "")'
    ws['D10'].number_format = '0.00%'
    ws.merge_cells('A12:A13')
    ws['A12'] = "模式二：快速稅額輸入"
    apply_styles_to_range(ws, 'A12', {
                          'font': font_section_header, 'fill': fill_header_grey, 'alignment': align_center})
    ws.merge_cells('B12:D12')
    ws.merge_cells('B13:D13')
    apply_styles_to_range(ws, 'B12:B13', {
                          'font': font_body, 'fill': fill_blue_input, 'alignment': align_center})
    ws['E12'] = "年度應納地價稅 (快速)"
    ws['E13'] = "年度應納房屋稅 (快速)"
    apply_styles_to_range(
        ws, 'E12:E13', {'font': font_body, 'alignment': align_left})
    ws.merge_cells('F12:F13')
    ws['F12'] = "若填寫此區藍色欄位，將優先採用此處稅額進行計算。"
    apply_styles_to_range(
        ws, 'F12', {'font': font_note, 'alignment': align_center})
    ws.merge_cells('A14:F14')
    ws['A14'] = "（二）核心稅費計算"
    apply_styles_to_range(ws, 'A14', {
                          'font': font_section_header, 'fill': fill_header_grey, 'alignment': align_center})
    for col_idx, header in enumerate(["項目", "賣方持有期間", "買方持有期間", "年度總稅額", "找補金額", "備註 (計算過程)"], 1):
        ws.cell(row=15, column=col_idx, value=header)
    apply_styles_to_range(ws, 'A15:F15', {
                          'font': font_body_bold, 'fill': fill_header_grey, 'alignment': align_center})
    ws['A16'] = "地價稅"
    ws['A17'] = "房屋稅"
    apply_styles_to_range(
        ws, 'A16:A17', {'font': font_body, 'alignment': align_center})
    apply_styles_to_range(ws, 'B16:C16', {
                          'font': font_body, 'alignment': align_center, 'fill': fill_period_land})
    apply_styles_to_range(ws, 'B17:C17', {
                          'font': font_body, 'alignment': align_center, 'fill': fill_period_house})
    apply_styles_to_range(
        ws, 'F16:F17', {'font': font_body, 'alignment': align_left, 'wrap_text': True})
    apply_styles_to_range(ws, 'D16:E17', {
                          'font': font_body, 'alignment': align_center, 'number_format': '#,##0'})
    ws.merge_cells('A19:F19')
    ws['A19'] = "（三）其他費用分算"
    apply_styles_to_range(ws, 'A19', {
                          'font': font_section_header, 'fill': fill_header_grey, 'alignment': align_center})
    ws['A20'] = "管理費(月)"
    ws['A21'] = "車位清潔費(月)"
    ws['A22'] = "押金/保證金"
    ws['A23'] = "管理費已預繳至"
    ws['A24'] = "車位清潔費已預繳至"
    apply_styles_to_range(ws, 'A20:A24', {
                          'font': font_body, 'fill': fill_header_grey, 'alignment': align_right})
    apply_styles_to_range(ws, 'B20:B24', {
                          'font': font_body, 'fill': fill_yellow_input, 'alignment': align_center})
    ws['B20'] = 4430
    ws['B21'] = 1200
    ws['B22'] = 0
    ws['B23'] = 1141031
    ws['B24'] = 1141031
    ws['C23'] = '=IF(AA6<>"", TEXT(AA6, "民國 e 年 m 月 d 日"), "格式: YYYMMDD")'
    ws['C24'] = '=IF(AA7<>"", TEXT(AA7, "民國 e 年 m 月 d 日"), "格式: YYYMMDD")'
    ws.merge_cells('C20:F21')
    ws['C20'] = "請填寫月繳金額，若無則填 0。\n找補金額將依交屋日與費用預繳截止日，按當月天數比例分算。"
    apply_styles_to_range(
        ws, 'C20', {'font': font_note, 'alignment': align_left})
    ws.merge_cells('E22:F22')
    ws['E22'] = "此金額將作為「賣方應付給買方」的款項"
    apply_styles_to_range(
        ws, 'E22', {'font': font_note, 'alignment': align_left})
    ws.merge_cells('D23:F23')
    ws.merge_cells('D24:F24')
    ws.merge_cells('A25:C25')
    ws['A25'] = "其他費用找補金額 (管理費、清潔費)"
    apply_styles_to_range(
        ws, 'A25', {'font': font_body_bold, 'alignment': align_center})
    ws.merge_cells('D25:F25')
    apply_styles_to_range(ws, 'D25', {
                          'font': font_body_bold, 'number_format': '#,##0', 'alignment': align_center})
    ws.merge_cells('A26:F26')
    ws['A26'] = '=IF(D25>0, "買方應補貼賣方 NT$ " & TEXT(D25, "#,##0") & " 元", "管理費、清潔費無需找補")'
    apply_styles_to_range(
        ws, 'A26', {'font': font_body, 'alignment': align_center})
    ws.merge_cells('A27:F27')
    ws['A27'] = "（四）最終結算"
    apply_styles_to_range(ws, 'A27', {
                          'font': font_section_header, 'fill': fill_header_grey, 'alignment': align_center})
    ws.merge_cells('A28:A29')
    ws['A28'] = "應付總計"
    apply_styles_to_range(
        ws, 'A28', {'font': font_body_bold, 'alignment': align_center})
    ws['B28'] = "買方應收總計"
    ws['C28'] = "賣方應收總計"
    apply_styles_to_range(ws, 'B28:C28', {
                          'font': font_body, 'alignment': align_center, 'fill': fill_header_grey})
    apply_styles_to_range(ws, 'B29:C29', {
                          'font': font_body_bold, 'number_format': '"NT$ "#,##0', 'alignment': align_center})
    ws.merge_cells('D28:F29')
    ws.merge_cells('A30:A31')
    ws['A30'] = "最終結算結果"
    apply_styles_to_range(ws, 'A30', {
                          'font': font_section_header, 'fill': fill_result_orange, 'alignment': align_center})
    ws.merge_cells('B30:F31')
    apply_styles_to_range(ws, 'B30', {
                          'font': font_final_result, 'fill': fill_result_orange, 'alignment': align_center})
    ws.merge_cells('A33:F33')
    ws['A33'] = "本計算機由 陳定康 設計"
    apply_styles_to_range(
        ws, 'A33', {'font': font_footer, 'alignment': align_right})

    # --- 備註與作者設定 ---
    comments = {
        'B2': '請輸入民國年日期。\n格式為 YYYMMDD 或 YYMMDD。\n範例：114年10月1日，請輸入 1141001。',
        'B3': '請輸入民國年日期。\n格式為 YYYMMDD 或 YYMMDD。\n範例：114年10月31日，請輸入 1141031。',
        'B4': '此選項將影響交屋當日費用歸屬：\n● 否 (預設)：交屋日歸【賣方】負責。\n● 是：交屋日歸【買方】負責。',
        # 【介面修改】為 B9 新增功能說明
        'B9': '請輸入房屋評定現值總額。\n\n【新功能】可使用減法格式輸入：\n[總現值]-[免稅騎樓現值]\n範例: 13272200-277300',
        'C7': '請輸入分數，例如：1/4。\n若此欄位與左側百分比欄位皆有數值，將【優先採用此分數】進行計算。',
        'B12': '若您已知曉年度總稅額，可直接填寫於此。\n系統將【優先採用此處稅額】，並忽略上方「模式一」的詳細資料。',
        'B13': '若您已知曉年度總稅額，可直接填寫於此。\n系統將【優先採用此處稅額】，並忽略上方「模式一」的詳細資料。',
        'B23': '請輸入費用已預繳之「截止日期」。\n格式為 YYYMMDD 或 YYMMDD。\n範例：預繳至114年10月31日，請輸入 1141031。',
        'B24': '請輸入費用已預繳之「截止日期」。\n格式為 YYYMMDD 或 YYMMDD。\n範例：預繳至114年10月31日，請輸入 1141031。'
    }
    for cell_ref, comment_text in comments.items():
        ws[cell_ref].comment = Comment(comment_text, "陳定康")

    # 備註區塊佈局與樣式
    ws.merge_cells('I1:N1')
    ws['I1'] = "【稅務資訊備註】"
    apply_styles_to_range(ws, 'I1', {'font': font_body_bold})
    ws.merge_cells('I2:N2')
    apply_styles_to_range(ws, 'I2', {'font': font_body_bold})
    ws.merge_cells('I3:N7')
    ws.merge_cells('I8:N8')
    apply_styles_to_range(ws, 'I8', {'font': font_body_bold})
    ws.merge_cells('I9:N13')
    apply_styles_to_range(
        ws, 'I3:I13', {'font': font_body, 'alignment': align_left, 'wrap_text': True})

    # 統一套用框線
    for r in ['A2:F4', 'A5:F10', 'A12:F13', 'A14:F18', 'A19:F26', 'A27:F31', 'I1:N13']:
        apply_styles_to_range(ws, r, {'border': thin_border})
    apply_styles_to_range(ws, 'A30:F31', {'border': medium_border})

    # --- 資料驗證 ---
    dv_land = DataValidation(type="list", formula1="=稅率表!$D$2:$D$3")
    dv_land.add('B8')
    ws.add_data_validation(dv_land)
    dv_house = DataValidation(type="list", formula1="=稅率表!$A$2:$A$14")
    dv_house.add('B10')
    ws.add_data_validation(dv_house)

    dv_date = DataValidation(type="custom", formula1="=AA2>=AA1")
    dv_date.errorTitle = "日期輸入錯誤"
    dv_date.error = "【房屋點交日期】必須晚於或等於【權狀登記日期】。\n\n請重新輸入正確的日期。"
    dv_date.add('B3')
    ws.add_data_validation(dv_date)

    # --- 建立稅率表工作表 ---
    ws2 = wb.create_sheet("稅率表")
    house_tax_data = [("房屋使用情境選項", "對應稅率"), ("自住用-全國單一自住(現值一定金額下)", 0.01), ("自住用-全國3戶內", 0.012), ("公益出租人/社會住宅", 0.012), ("非自住-出租(達租金標準)/繼承共有-4戶內", 0.015), ("非自住-出租(達租金標準)/繼承共有-5~6戶", 0.02),
                      ("非自住-出租(達租金標準)/繼承共有-7戶以上", 0.024), ("非自住-其他住家用-2戶內", 0.032), ("非自住-其他住家用-3~4戶", 0.038), ("非自住-其他住家用-5~6戶", 0.042), ("非自住-其他住家用-7戶以上", 0.048), ("營業用", 0.03), ("私人醫院/診所/事務所用", 0.03), ("非住家非營業用(人民團體等)", 0.02)]
    land_tax_data = [("地價稅率選項", "對應稅率"), ("自用住宅 (千分之二)",
                                          0.002), ("一般用地 (千分之十)", 0.010)]
    for r_idx, row_data in enumerate(house_tax_data, 1):
        for c_idx, cell_data in enumerate(row_data, 1):
            ws2.cell(row=r_idx, column=c_idx, value=cell_data)
    for r_idx, row_data in enumerate(land_tax_data, 1):
        for c_idx, cell_data in enumerate(row_data, 1):
            ws2.cell(row=r_idx, column=c_idx + 3, value=cell_data)
    apply_styles_to_range(ws2, 'A1:B1', {'font': font_body_bold})
    apply_styles_to_range(ws2, 'D1:E1', {'font': font_body_bold})
    apply_styles_to_range(ws2, 'B2:B14', {'number_format': '0.00%'})
    apply_styles_to_range(ws2, 'E2:E3', {'number_format': '0.000%'})
    ws2.column_dimensions['A'].width = 45
    ws2.column_dimensions['B'].width = 15
    ws2.column_dimensions['D'].width = 30
    ws2.column_dimensions['E'].width = 15

    # --- 儲存檔案 ---
    try:
        wb.calculation.fullCalcOnLoad = True
        wb.save(filename)
        print(f"✅ 成功創建 Excel 檔案： '{filename}'")
        print("-" * 50)
        print("💡 新功能提示：")
        print("   - 房屋現值(B9)支援減法格式，備註(F17)將自動顯示計算過程。")
        print("-" * 50)

    except Exception as e:
        print(f"❌ 創建檔案時發生錯誤：{e}")


if __name__ == '__main__':
    create_final_workbook_v5_12()

✅ 成功創建 Excel 檔案： '交屋稅費計算機(v5.12_房屋現值優化版).xlsm'
--------------------------------------------------
💡 新功能提示：
   - 房屋現值(B9)支援減法格式，備註(F17)將自動顯示計算過程。
--------------------------------------------------
